In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from pprint import pprint
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain import hub

## Multi Query

### Web文書の取得、分割、リトリーバ作成

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)
blog_docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(blog_docs)

In [ ]:
vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

### マルチクエリプロンプト作成

In [ ]:
# 異なる視点を得る為のマルチクエリプロンプト
# あなたはAI言語モデルアシスタントです。あなたの仕事は、ベクトルデータベースから関連する文書を取得するために、
# 与えられたユーザーの質問を5つの異なるバージョンで生成することです。
# ユーザーの質問に対する複数の視点を生成することにより、距離ベースの類似性検索の一部の限界を克服する助けとなることがあなたの目標です。
# 代替となる質問を改行で区切って提供してください。元の質問: {question}
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

### マルチクエリチェーンの作成

In [ ]:
generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

### マルチクエリによる文書検索

In [ ]:
def get_unique_union(documents: list[list]):

    # 各種要素数確認
    print("親要素数:", len(documents))
    print("子要素数:", [len(sublist) for sublist in documents])

    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]

In [ ]:
question = "What is task decomposition for LLM agents?"

In [ ]:
def print_list_and_pass(x):
    print('要素数:', len(x))
    pprint(x)
    return x

retrieval_chain = generate_queries | print_list_and_pass | retriever.map() | get_unique_union # 作成したマルチクエリの確認を含めるチェーン
# retrieval_chain = generate_queries | retriever.map() | get_unique_union # 
docs = retrieval_chain.invoke({'question': question})
print('文書数:', len(docs))
pprint(docs)

### 回答生成

In [ ]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

## RAG-Fusion

### マルチクエリプロンプト作成

In [ ]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

### RAG-Fusionリランキング処理

In [ ]:
def reciprocal_rank_fusion(results: list[list], k=60):

    # 各種要素数確認
    print("親要素数:", len(results))
    print("子要素数:", [len(sublist) for sublist in results])

    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)


    reranked_results = [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x:x[1], reverse=True)
    ]
    return reranked_results # 定数あるいは引数を指定してここで上位のもののみ取得できるように絞り込みを行う実装も可能

In [ ]:
retrieval_chain_rag_fusion = generate_queries | print_list_and_pass | retriever.map()  | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({'question': question})
print('文書数:', len(docs))
pprint(docs)

### 回答生成

In [ ]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

## Decomposition

### Decompositionプロンプト作成

In [ ]:
# 与えられた問題を部分問題に分解する為のDecompositionプロンプト
# あなたは入力された質問に関連する複数のサブ質問を生成する役立つアシスタントです。
# 目標は、入力を分離して回答可能な一連のサブ問題／サブ質問に分解することです。
# 次の質問に関連する複数の検索クエリを生成してください: {question}
# 出力（3つのクエリ）：
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries separated by new lines):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

### Decompositionチェーン作成

In [ ]:
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
# Decomposition実行
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({'question': question})
pprint(questions)

### Decompositonで作成された質問の回答生成

#### Decomposition作成質問を逐次追加するパターン

In [ ]:
# ここに回答しないといけない質問があります：
#\n --- \n {question} \n --- \n
# ここに利用可能な背景の質問と回答のペアがあります：
# \n --- \n {q_a_pairs} \n --- \n
# 質問に関連する追加のコンテキストがここにあります：
# \n --- \n {context} \n --- \n
# 上記のコンテキストと背景の質問＋回答のペアを使用して、質問に答えてください：\n {question}
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""
decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# 質問回答ペア文字列作成
def format_qa_pair(question, answer):
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

In [ ]:
# 回答生成
q_a_pairs = ''
for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n---\n"+ q_a_pair
pprint(answer)

#### Decomposition作成質問を個別処理するパターン

In [ ]:
prompt_rag = hub.pull("rlm/rag-prompt")

# サブ質問のリストと対応する回答のリストを作成する処理
def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):

    sub_questions = sub_question_generator_chain.invoke({'question': question})
    rag_results = []

    for sub_question in sub_questions:

        retrieved_docs = retriever.get_relevant_documents(sub_question)

        answer = (
            prompt_rag | llm | StrOutputParser()
        ).invoke({'context': retrieved_docs, 'question': sub_question})
        rag_results.append(answer)

    return rag_results, sub_questions

In [ ]:
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)
for temp_q, temp_a in zip(questions, answers):
    print(f"Question: {temp_q}\nAnswer: {temp_a}\n")

In [ ]:
# 複数質問回答ペア文字列作成処理
def format_qa_pairs(questions, answers):
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

In [ ]:
context = format_qa_pairs(questions, answers)

template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

pprint(final_rag_chain.invoke({"context":context, "question":question}))

## Step Back

### Step Backプロンプトの作成

In [ ]:
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("system", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# あなたは世界の知識に精通しています。
# あなたの仕事は、一歩引いて質問をより一般的な「一歩引いた質問」に言い換えることであり、それによって質問を回答しやすくすることです。
# 以下にいくつかの例を示します：
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",),
        few_shot_prompt,
        ('user', '{question}'),
    ]
)

In [ ]:
def print_messages_and_pass_(x):
    pprint(x.messages)
    return x
# generate_queries_step_back = prompt | print_messages_and_pass_ | ChatOpenAI(temperature=0) | StrOutputParser()
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
print(generate_queries_step_back.invoke({"question": question}))

### 回答作成

In [ ]:
# あなたは世界知識の専門家です。これから質問をします。
# 回答は包括的に、かつ与えられた文脈と関連性がある場合は矛盾しないようにしてください。関連性がなければ無視してください。
# 通常(元質問)の文脈
# 抽象化した質問の文脈
# 元質問: {question}
# 回答:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

print(chain.invoke({"question": question}))

### HyDE(Hypothetical Document Embedding)

In [ ]:
# 次の質問に答えるための科学論文の一説を書いてください
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

In [ ]:
# 仮説的な回答生成
generate_docs_for_retrieval = (
    prompt_hyde
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

question = "What is task decomposition for LLM agents?"
print(generate_docs_for_retrieval.invoke({"question":question}))

In [ ]:
# 仮説的回答に似た文書の検索
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question": question})
pprint(retrieved_docs)

### 回答生成

In [ ]:
# 複数文書結合処理
def unify_docs(documents: list):
    unified_str = '\n---\n'.join([document.page_content for document in documents])
    return unified_str

In [ ]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": RunnableLambda(lambda x: unify_docs(x["context"])), "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"context":retrieved_docs, "question":question}))